#### Frame extraction

In [ ]:
import annotationlib
import os
import cv2
from glob import glob

output_dir = "data/cctv_frames"
os.makedirs(output_dir, exist_ok=True)

videos_directory = "data/cctv_videos/"
videos = glob(videos_directory + "*.mp4")

# Extract frames and export to data/cctv_frames/
for video in videos:
    # Get the filename without extension to use as a prefix
    video_name = os.path.splitext(os.path.basename(video))[0]
    
    cap = cv2.VideoCapture(video)
    count = 0
    
    print(f"Processing {video_name}...")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame as JPEG
        frame_filename = os.path.join(output_dir, f"{video_name}_frame_{count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        count += 1
        
        if count % 500 == 0:
            print(f"  Extracted {count} frames...")
            
    cap.release()
    print(f"Finished. Total frames extracted from {video_name}: {count}")

Divide into a split of 2 for annotation

In [3]:
import shutil
from tqdm import tqdm

frames = glob(output_dir + "/*.jpg")

mid = len(frames) // 2
for i, f in tqdm(enumerate(frames)):
    subdir = "aditya" if i < mid else "nitish"
    dest_dir = os.path.join(output_dir, subdir)
    os.makedirs(dest_dir, exist_ok=True)
    shutil.move(f, os.path.join(dest_dir, os.path.basename(f)))

17251it [00:11, 1496.82it/s]


#### Restructure annotations for Anomaly-Videos-Part-3

In [33]:
import os
import shutil
from pathlib import Path

root_data = Path("../data/individual_sets/all_images_anomaly")
image_dir = root_data / "images"
labels_dir = root_data / "labels"

# image_dir.mkdir(parents=True, exist_ok=True)
# labels_dir.mkdir(parents=True, exist_ok=True)

# for file_path in root_data.iterdir():
#     if file_path.is_file() and file_path.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
#         shutil.move(file_path, image_dir / file_path.name)

In [50]:
import json

annotations_file = "../temp/annotations_all.json"
width = 320
height = 240

with open(annotations_file, "r") as f:
    annotations = json.load(f)

images = sorted([image_dir / Path(image["file_name"].split(".")[0] + ".png") 
          for image in annotations["images"]])

anns = annotations["annotations"]

# Group annotations by image_id
img_to_anns = {}
for ann in anns:
    img_id = ann['image_id']
    if img_id not in img_to_anns:
        img_to_anns[img_id] = []
    img_to_anns[img_id].append((ann['bbox'], ann['category_id'] - 1))  # Convert to 0-indexed mapping as per data/all_images/class_dict.json

for img_id, anns in img_to_anns.items():
    label_path = labels_dir / Path(str(img_id).zfill(4) + ".txt")
    with open(label_path, "w") as f:
        for ann in anns:
            x1, y1, w, h = ann[0]
            # print(x1, y1, w, h)
            x_center = round((x1 + w/2) / width, 6)
            y_center = round((y1 + h/2) / height, 6)
            w = round(w / width, 6)
            h = round(h / height, 6)
            print(img_id, ann, width, height, x_center, y_center, w, h)
            f.write(f"{ann[1]} {x_center} {y_center} {w} {h}\n")

1335 ([188, 136, 11, 12], 1) 320 240 0.604688 0.591667 0.034375 0.05
1335 ([173, 104, 48, 57], 0) 320 240 0.615625 0.552083 0.15 0.2375
1336 ([189, 135, 12, 12], 1) 320 240 0.609375 0.5875 0.0375 0.05
1336 ([173, 101, 50, 59], 0) 320 240 0.61875 0.54375 0.15625 0.245833
1336 ([190, 173, 65, 36], 0) 320 240 0.695312 0.795833 0.203125 0.15
1337 ([187, 135, 10, 12], 1) 320 240 0.6 0.5875 0.03125 0.05
1337 ([172, 102, 46, 56], 0) 320 240 0.609375 0.541667 0.14375 0.233333
1337 ([182, 166, 75, 44], 0) 320 240 0.685937 0.783333 0.234375 0.183333
1338 ([185, 133, 8, 9], 1) 320 240 0.590625 0.572917 0.025 0.0375
1338 ([169, 100, 48, 59], 0) 320 240 0.603125 0.539583 0.15 0.245833
1338 ([177, 161, 48, 49], 0) 320 240 0.628125 0.772917 0.15 0.204167
1339 ([115, 151, 7, 15], 1) 320 240 0.370312 0.660417 0.021875 0.0625
1339 ([106, 117, 43, 42], 0) 320 240 0.398438 0.575 0.134375 0.175
1340 ([113, 148, 9, 17], 1) 320 240 0.367188 0.652083 0.028125 0.070833
1340 ([107, 116, 40, 55], 0) 320 240 0.39

#### Quickdraw on data/individual_sets/cctv-weapon-dataset images using labels

In [57]:
import cv2
import os
from glob import glob

image_dir = '../data/individual_sets/cctv-weapon-dataset/images'
label_dir = '../data/individual_sets/cctv-weapon-dataset/labels'

output_dir = '../data/individual_sets/cctv-weapon-dataset/drawn'
os.makedirs(output_dir, exist_ok=True)

count = 0
for img_path, label_path in zip(glob(f"{image_dir}/*"), glob(f"{label_dir}/*")):
    if not os.path.exists(img_path) or not os.path.exists(label_path):
        continue

    img = cv2.imread(img_path)
    h, w, _ = img.shape
    with open(label_path, 'r') as file:
        text = file.readlines()
        if text == []:
            continue
        count += 1
        for line in text:
            print(line)
            parts = line.strip().split()
            print(parts)
            if len(parts) < 5:
                continue
            # YOLO format: class x_center y_center width height (normalized)
            class_id = parts[0]
            _, x_c, y_c, bw, bh = map(float, parts)
            
            x1 = max(0, int((x_c - bw / 2) * w))
            y1 = max(0, int((y_c - bh / 2) * h))
            x2 = min(w, int((x_c + bw / 2) * w))
            y2 = min(h, int((y_c + bh / 2) * h))
            
            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw class ID text
            # Ensure text position is within image boundaries (at least 20px from top)
            text_y = max(y1 - 10, 20)
            cv2.putText(img, class_id, (x1, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imwrite(os.path.join(output_dir, os.path.basename(img_path)), img)
    if count == 10:
        break

0 0.688909 0.486803 0.114789 0.317230

['0', '0.688909', '0.486803', '0.114789', '0.317230']
1 0.642249 0.448207 0.080718 0.086321
['1', '0.642249', '0.448207', '0.080718', '0.086321']
0 0.343196 0.529880 0.152173 0.302789

['0', '0.343196', '0.529880', '0.152173', '0.302789']
1 0.393479 0.528552 0.033081 0.037185
['1', '0.393479', '0.528552', '0.033081', '0.037185']
0 0.466910 0.529139 0.145428 0.326003

['0', '0.466910', '0.529139', '0.145428', '0.326003']
1 0.526465 0.466135 0.039697 0.034529
['1', '0.526465', '0.466135', '0.039697', '0.034529']
0 0.504826 0.540505 0.134037 0.297477

['0', '0.504826', '0.540505', '0.134037', '0.297477']
1 0.559546 0.434263 0.042344 0.029216
['1', '0.559546', '0.434263', '0.042344', '0.029216']
0 0.609168 0.458167 0.099243 0.302789

['0', '0.609168', '0.458167', '0.099243', '0.302789']
1 0.554915 0.418327 0.088657 0.053121
['1', '0.554915', '0.418327', '0.088657', '0.053121']
0 0.806331 0.501992 0.083364 0.316069

['0', '0.806331', '0.501992', '0.083

In [58]:
import os
import shutil
import random
from pathlib import Path
from tqdm import tqdm

# Define paths
# Assuming running from 'ml' directory or relative to it
base_dir = Path('../data/merged_dataset/complete_set')
if not base_dir.exists():
    # If running from project root
    base_dir = Path('data/merged_dataset')

images_dir = base_dir / 'images'
labels_dir = base_dir / 'labels'

if not images_dir.exists():
    print(f"Error: {images_dir} does not exist")
    exit(1)

# Define split directories
splits = ['train', 'val', 'test']
for split in splits:
    for dtype in ['images', 'labels']:
        (base_dir / split / dtype).mkdir(parents=True, exist_ok=True)

# Get all images
image_extensions = {'.png', '.jpg', '.jpeg', '.bmp'}
images = [f for f in images_dir.iterdir() if f.suffix.lower() in image_extensions]

# Shuffle images
random.seed(42)  # For reproducibility
random.shuffle(images)

# Calculate split counts
total_images = len(images)
train_count = int(total_images * 0.75)
val_count = int(total_images * 0.15)
test_count = total_images - train_count - val_count

# Split data
train_imgs = images[:train_count]
val_imgs = images[train_count:train_count + val_count]
test_imgs = images[train_count + val_count:]

def copy_files(file_list, destination_split):
    dest_img_dir = base_dir / destination_split / 'images'
    dest_lbl_dir = base_dir / destination_split / 'labels'
    
    for img_path in tqdm(file_list, desc=f"Copying {destination_split}"):
        # Copy image
        shutil.copy2(img_path, dest_img_dir / img_path.name)
        
        # Copy label if exists
        label_path = labels_dir / (img_path.stem + '.txt')
        if label_path.exists():
            shutil.copy2(label_path, dest_lbl_dir / label_path.name)

print(f'Total images: {total_images}')
print(f'Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}')

# Execute copy
print("Starting copy process...")
copy_files(train_imgs, 'train')
copy_files(val_imgs, 'val')
copy_files(test_imgs, 'test')
print('Done!')


Total images: 1737
Train: 1302, Val: 260, Test: 175
Starting copy process...


Copying test: 100%|██████████| 175/175 [00:02<00:00, 83.19it/s]

Done!


Create empty annotation files as needed

In [55]:
import shutil

images = glob('../data/merged_dataset/images/*')
labels = glob('../data/merged_dataset/labels/*')
missing_dir = "../data/merged_dataset/missing_annotation/"
os.makedirs(missing_dir, exist_ok=True)

for label in labels:
    image = label.replace('labels', 'images').replace('.txt', '.png')
    # label = image.replace('images', 'labels').replace('.png', '.txt')
    if not os.path.exists(image):
        print(label)
        shutil.move(label, missing_dir)

../data/merged_dataset/labels\1511.txt


In [29]:
from glob import glob

all_cams = glob("../data/merged_dataset/**/Cam*", recursive=True)

for file in all_cams:
    os.remove(file)

#### Model conversion

In [75]:
from ultralytics import YOLO

model = YOLO("../services/stream-worker/src/model/yolov8m.pt") 

results = model.train(
    data='../data/merged_dataset/data.yaml',    # Path to your data config
    epochs=5,                   # Adjust based on dataset size
    imgsz=640,                  # Input resolution
    batch=0.8,                   # Reduce if you hit GPU memory limits
    patience=25,                # Early stopping if no improvement
    device=0,                   # GPU index (or 'cpu' if no GPU)
    workers=0,                  # DataLoader workers
    project='runs/train',       # Output directory
    name='test_v1',             # Experiment name
    exist_ok=True,              # Overwrite if exists
    rect=True
    
    # # Data augmentation (enabled by default, but you can tune)
    # augment=True,
    # hsv_h=0.015,              # Hue augmentation
    # hsv_s=0.7,                # Saturation augmentation
    # hsv_v=0.4,                # Value/brightness augmentation
    # degrees=10.0,             # Rotation range
    # scale=0.5,                # Scale augmentation
    # flipud=0.0,               # Vertical flip (usually 0 for surveillance)
    # fliplr=0.5,               # Horizontal flip
    # mosaic=1.0,               # Mosaic augmentation probability
)

print("Pipeline Test Complete")

New https://pypi.org/project/ultralytics/8.4.5 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.248  Python-3.11.14 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5060 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../services/stream-worker/src/model/yolov8m.pt, momentum=0.937, mosaic=1.0, mul

KeyboardInterrupt: 

In [ ]:
# Here
